# Analysis and Sampling of Molecular Simulations by adversarial Autoencoders
---
1. [Packages import](#1.-Packages-import)
2. [Internal coordinates computation](#2.-Internal-coordinates-computation)
3. [Execution & visualization](#3.-Execution-&-visualization)

## 1. Packages import

In [ ]:
# Import packages

from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
from src import asmsa
from src.gan import GAN
from src.visualizer import GAN_visualizer
import mdtraj as md
import numpy as np
import nglview as nv
import urllib.request

In [ ]:
# Define input files
%cd ~

# input conformation
#conf = "alaninedipeptide_H.pdb"
conf = "trpcage_correct.pdb"

# input trajectory
# atom numbering must be consistent with {conf}

#traj = "alaninedipeptide_reduced.xtc"
traj = "trpcage_red.xtc"

# input topology
# expected to be produced with 
#    gmx pdb2gmx -f {conf} -p {topol} -n {index} 

# Gromacs changes atom numbering, the index file must be generated and used as well

#topol = "topol.top"
topol = "topol_correct.top"
index = 'index_correct.ndx'

## 2. Internal coordinates computation

In [ ]:
tr = md.load(traj,top=conf)
idx=tr[0].top.select("name CA")
#idx=tr[0].top.select("element != H")
tr.superpose(tr[0],atom_indices=idx)
geom = np.moveaxis(tr.xyz ,0,-1)

In [ ]:
v = nv.show_mdtraj(tr)
v.clear()
v.add_representation("licorice")
v

In [ ]:
geom.shape

In [ ]:
# Define sparse and dense feture extensions of IC
density = 2 # integer in [1, n_atoms-1]
sparse_dists = asmsa.NBDistancesSparse(geom.shape[0], density=density)
dense_dists = asmsa.NBDistancesDense(geom.shape[0])

# mol = asmsa.Molecule(conf,topol)
# mol = asmsa.Molecule(conf,topol,fms=[sparse_dists])
mol = asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[sparse_dists])

In [ ]:
X_train = mol.intcoord(geom).T
X_train.shape

## 3. Set visualizer

In [ ]:
visualizer = GAN_visualizer(
                    nbins=50,
                    visualize_freq=10,
#                     analysis_files=['visualization/rama_ala_reduced0.txt',
#                                     'visualization/rama_ala_reduced1.txt',
#                                     'visualization/angever0.txt',
#                                     'visualization/angever1.txt',
#                                     'visualization/angever2.txt'],
                    figsize=(15,5),
                    cmap='hsv'
                )

In [ ]:
# download mushroom image as prior
urllib.request.urlretrieve("https://drive.google.com/uc?export=download&id=1I2WP92MMWS5s5vin_4cvmruuV-1W77Hl", "mushroom_bw.png")

In [ ]:
output_file = 'lows.txt'

gan = GAN(X_train,batch_size=256,prior='mushroom_bw.png')
test = gan.train(epochs=50, out_file=output_file, visualizer=visualizer) 

In [ ]:
visualizer = GAN_visualizer(
                    lows='lows.txt',
                    nbins=50,
                    figsize=(15,5),
                )

In [ ]:
visualizer.make_visualization()

In [ ]:
# Rgyr color coded in low dim (rough view)

lows = np.loadtxt(output_file)
rg = md.compute_rg(tr)
cmap = plt.get_cmap('rainbow')
plt.figure(figsize=(12,12))
plt.scatter(lows[:,0],lows[:,1],marker='.',c=rg,cmap=cmap)
plt.colorbar(cmap=cmap)
plt.show()